# Hidden Layer Quickstart

This notebook verifies your environment and demonstrates the core harness APIs.
Feel free to edit and save under a new name once everything is working.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# SETUP: Imports and Provider Detection
# ═══════════════════════════════════════════════════════════════════════
# Don't edit this cell - just run it to see what's available

import sys
import os
import shutil
from datetime import datetime

# Add paths for imports
sys.path.append('../../../')  # Go up to repo root (for harness)
sys.path.append('../')        # Go up to multi-agent/ (for multi_agent module)

# Import multi-agent specific functions
from multi_agent import run_strategy, STRATEGIES

# Import core harness functionality
from harness import ExperimentConfig, ExperimentResult, get_tracker, llm_call
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

print("✓ Imports successful\n")

# Detect what providers are available
available_providers = {}

# Check MLX (Apple Silicon local inference)
try:
    import mlx.core
    available_providers['mlx'] = {
        'name': 'MLX (Apple Silicon)',
        'default_model': 'mlx-community/Llama-3.2-3B-Instruct-4bit',
        'available': True
    }
except ImportError:
    available_providers['mlx'] = {'name': 'MLX (Apple Silicon)', 'available': False}

# Check Ollama (local inference)
if shutil.which("ollama"):
    available_providers['ollama'] = {
        'name': 'Ollama (local)',
        'default_model': 'llama3.2:latest',
        'available': True
    }
else:
    available_providers['ollama'] = {'name': 'Ollama (local)', 'available': False}

# Check Anthropic API
if os.getenv("ANTHROPIC_API_KEY"):
    available_providers['anthropic'] = {
        'name': 'Anthropic Claude',
        'default_model': 'claude-3-5-sonnet-20241022',
        'available': True
    }
else:
    available_providers['anthropic'] = {'name': 'Anthropic Claude', 'available': False}

# Check OpenAI API
if os.getenv("OPENAI_API_KEY"):
    available_providers['openai'] = {
        'name': 'OpenAI',
        'default_model': 'gpt-4o',
        'available': True
    }
else:
    available_providers['openai'] = {'name': 'OpenAI', 'available': False}

# Print what's available
print("Available Providers:")
for key, info in available_providers.items():
    status = "✓" if info['available'] else "✗"
    model_info = f" (default: {info['default_model']})" if info.get('default_model') else ""
    print(f"  {status} {info['name']}{model_info}")

print("\n" + "="*70)
print("Next: Edit the configuration cell below to choose your settings")
print("="*70)


In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# ⚙️  CONFIGURATION - EDIT THIS CELL TO CHANGE SETTINGS
# ═══════════════════════════════════════════════════════════════════════

# 1. Choose your provider (pick one that shows ✓ above)
PROVIDER = 'mlx'  # Options: 'mlx', 'ollama', 'anthropic', 'openai'

# 2. Choose your model (or None to use the default)
MODEL = None  # Examples:
              # MLX: 'mlx-community/Llama-3.2-3B-Instruct-4bit'
              # Ollama: 'llama3.2:latest', 'qwen2.5:latest'
              # Anthropic: 'claude-3-5-sonnet-20241022'
              # OpenAI: 'gpt-4o'

# 3. Hyperparameters
TEMPERATURE = 0.7      # Range: 0.0 (focused) to 1.0 (creative)
MAX_TOKENS = 2048      # Maximum length of response
SYSTEM_PROMPT = None   # Optional: role/persona for the model

# ═══════════════════════════════════════════════════════════════════════

# Validate and set defaults
if PROVIDER not in available_providers or not available_providers[PROVIDER]['available']:
    available = [k for k, v in available_providers.items() if v['available']]
    raise ValueError(f"Provider '{PROVIDER}' is not available. Choose from: {available}")

if MODEL is None:
    MODEL = available_providers[PROVIDER]['default_model']

# Print configuration
print("Current Configuration:")
print("="*70)
print(f"📍 Provider: {PROVIDER}")
print(f"🤖 Model: {MODEL}")
print(f"🌡️  Temperature: {TEMPERATURE}")
print(f"📏 Max Tokens: {MAX_TOKENS}")
print(f"💬 System Prompt: {SYSTEM_PROMPT or '(none)'}")
print("="*70)


In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# SMOKE TEST: Verify the LLM provider works
# ═══════════════════════════════════════════════════════════════════════

try:
    response = llm_call(
        "Say 'Hidden Layer online.'",
        provider=PROVIDER,
        model=MODEL,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        system_prompt=SYSTEM_PROMPT
    )
    
    print("✓ LLM call successful!")
    print(f"Model: {response.model}")
    print(f"Latency: {response.latency_s:.2f}s")
    print(f"Response: {response.text.strip()}\n")
    
except Exception as exc:
    print(f"✗ LLM call failed!")
    print(f"Provider: {PROVIDER}")
    print(f"Model: {MODEL}")
    print(f"Error: {exc}")
    raise


In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# STRATEGY TEST: Run a multi-agent strategy
# ═══════════════════════════════════════════════════════════════════════

strategy_result = run_strategy(
    "single",  # Strategy: 'single', 'debate', 'consensus', 'self_consistency', etc.
    "List two reasons multi-agent strategies can outperform single models.",
    provider=PROVIDER,
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    verbose=False
)

print("✓ Strategy executed successfully!")
print(f"Strategy: {strategy_result.strategy_name}")
print(f"Latency: {strategy_result.latency_s:.2f}s")
print(f"Output:\n{strategy_result.output.strip()}\n")


In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# EXPERIMENT TRACKING: Log results
# ═══════════════════════════════════════════════════════════════════════

config = ExperimentConfig(
    experiment_name="quickstart_smoke",
    task_type="demo",
    strategy="single",
    provider=PROVIDER,
    model=MODEL,
)

tracker = get_tracker()
run_dir = tracker.start_experiment(config)

result_record = ExperimentResult(
    config=config,
    task_input="Quickstart notebook demo task",
    output=strategy_result.output,
    latency_s=strategy_result.latency_s,
    tokens_in=strategy_result.tokens_in,
    tokens_out=strategy_result.tokens_out,
    cost_usd=strategy_result.cost_usd,
    eval_scores={},
    eval_metadata={},
    success=True,
    error=None,
)

tracker.log_result(result_record)
summary = tracker.finish_experiment()

print("✓ Experiment logged!")
print(f"Run directory: {run_dir}")
if summary:
    print(f"Total runs: {summary.get('total_runs')}")
    print(f"Average latency: {summary.get('avg_latency_s', 0):.2f}s")
    print(f"Timestamp: {summary['timestamp']}")
